# Feature postprocessing from paper.
- Title:  "Predicting yeast synthetic lethal genetic interactions using protein domains"
- Authors: Bo Li, Feng Luo,School of Computing,Clemson University,Clemson, SC, USA
- e-mail: bol, luofeng@clemson.edu
- year:2009

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict 
import seaborn as sns
import matplotlib.cm as cm
import scipy as scipy
import random

### Importing datasets 

#### Link to the github repo where the datasets to be downloaded:
[DOWNLOAD THE DATASETS HERE](https://github.com/leilaicruz/machine-learning-for-yeast/tree/dev_Leila/datasets-for-learning)

In [2]:
import os
script_dir = os.path.dirname('__file__') #<-- absolute dir the script is in
rel_path_SL = "datasets/data-synthetic-lethals.xlsx"
rel_path_nSL="datasets/data-positive-genetic.xlsx"
rel_path_domains="datasets/proteins-domains-from-Pfam.xlsx"

abs_file_path_SL = os.path.join(script_dir, rel_path_SL)
abs_file_path_nSL = os.path.join(script_dir, rel_path_nSL)
abs_file_path_domains = os.path.join(script_dir, rel_path_domains)

# os.chdir('mini_book/docs/') #<-- for binder os.chdir('../')
# os.chdir('../')
my_path_sl= abs_file_path_SL
my_path_non_sl=abs_file_path_nSL
my_path_domains=abs_file_path_domains

data_sl=pd.read_excel(my_path_sl,header=0)
data_domains=pd.read_excel(my_path_domains,header=0,index_col='Unnamed: 0')
data_domains=data_domains.dropna()
data_nonsl=pd.read_excel(my_path_non_sl,header=0)

## Building the feature matrix
One matrix for true SL where each row is one pair of SL. Every raw will be a vector of 0,1 or 2 depending on the comparison with the domain list. For row i the jth element = 0 if the jth element of the domain list is not in neither protein A and B, 1, if it is in one of them and 2 if it is in both of them .

### Building the list of proteins domains id per protein pair separately :
- List of protein A: Search for the Sl/nSL database the query gene name and look in the protein domain database which protein domains id has each of those queries.
- List of protein B: Search for the Sl/nSL database the target gene name of the previous query and look in the protein domain database which protein domains id has each of those target genes.

In [3]:
# Selecting the meaningful columns in the respective dataset
domain_id_list=data_domains['domain-name']
query_gene=data_sl['gene-query-name']
target_gene=data_sl['gene-target-name']
query_gene_nonlethal=data_nonsl['gene-query-name']
target_gene_nonlethal=data_nonsl['gene-target-name']



# Initialising the arrays
protein_a_list=[]
protein_b_list=[]
protein_a_list_non=[]
protein_b_list_non=[]

population = np.arange(0,len(data_sl))

# For loop for 10000 pairs sampled randomly from the SL/nSl pair list , and creating a big array of proteind domains id per protein pair

for m in random.sample(list(population), 500):
    protein_a=data_domains[data_domains['name']==query_gene[m]]
    protein_b=data_domains[data_domains['name']==target_gene[m]]
    protein_a_list.append(protein_a['domain-name'].tolist())
    protein_b_list.append(protein_b['domain-name'].tolist())

    protein_a_non=data_domains[data_domains['name']==query_gene_nonlethal[m]]
    protein_b_non=data_domains[data_domains['name']==target_gene_nonlethal[m]]
    protein_a_list_non.append(protein_a_non['domain-name'].tolist())
    protein_b_list_non.append(protein_b_non['domain-name'].tolist())

In [4]:
print('We are going to analyze',len((protein_a_list)) ,'protein pairs, out of',len(data_sl),'SL protein pairs')
print('We are going to analyze',len((protein_a_list_non)) ,'protein pairs, out of',len(data_nonsl),'positive protein pairs')

We are going to analyze 500 protein pairs, out of 17871 SL protein pairs
We are going to analyze 500 protein pairs, out of 43340 positive protein pairs


### Postprocessing #1:  Remove protein pairs from study if either protein in the pair does not contain any domain

In [5]:
def remove_empty_domains(protein_list_search,protein_list_pair):
    index=[]
    for i in np.arange(0,len(protein_list_search)):
        if protein_list_search[i]==[] or protein_list_pair[i]==[]:
            index.append(i) ## index of empty values for the protein_a_list meaning they dont have any annotated domain

    y=[x for x in np.arange(0,len(protein_list_search)) if x not in index] # a list with non empty values from protein_a list

    protein_list_search_new=[]
    protein_list_pair_new=[]
    for i in y:
        protein_list_search_new.append(protein_list_search[i])
        protein_list_pair_new.append(protein_list_pair[i])
    return protein_list_search_new,protein_list_pair_new

## evaluating the function

protein_a_list_new,protein_b_list_new=remove_empty_domains(protein_a_list,protein_b_list)

protein_a_list_non_new,protein_b_list_non_new=remove_empty_domains(protein_a_list_non,protein_b_list_non)

In [6]:
print('The empty domain in the SL were:', len(protein_a_list)-len(protein_a_list_new), 'out of', len(protein_a_list),'domains')
print('The empty domain in the nSL were:', len(protein_a_list_non)-len(protein_a_list_non_new), 'out of', len(protein_a_list_non),'domains')

The empty domain in the SL were: 85 out of 500 domains
The empty domain in the nSL were: 101 out of 500 domains


### Feature engineering: Select from each ordered indexes of domain id list which of them appear once, in both or in any of the domains of each protein pair

#### Define function `get_indexes`

In [7]:
get_indexes = lambda x, xs: [i for (y, i) in zip(xs, range(len(xs))) if x == y] # a function that give the index of whether a value appear in array or not
a=[1,2,2,4,5,6,7,8,9,10]
get_indexes(2,a)

[1, 2]

In [8]:
def feature_building(protein_a_list_new,protein_b_list_new):
    x = np.unique(domain_id_list)
    ## To avoid taking repeated domains from one protein of the pairs , lets reduced the domains of each protein from the pairs to their unique members
    protein_a_list_unique=[]
    protein_b_list_unique=[]
    for i in np.arange(0,len(protein_a_list_new)):
        protein_a_list_unique.append(np.unique(protein_a_list_new[i]))
        protein_b_list_unique.append(np.unique(protein_b_list_new[i]))
        
    protein_feat_true=np.zeros(shape=(len(x),len(protein_a_list_unique)))
    pair_a_b_array=[]
    for i in np.arange(0,len(protein_a_list_unique)):
        index_a=[]
        
        pair=[protein_a_list_unique[i],protein_b_list_unique[i]]
        pair_a_b=np.concatenate(pair).ravel()
        pair_a_b_array.append(pair_a_b)

    j=0
    for i in pair_a_b_array:  
        array,index,counts=np.unique(i,return_index=True,return_counts=True)
        
        for k,m in zip(counts,array):
            if k ==2:
                protein_feat_true[get_indexes(m,x),j]=2
                
            if k==1:
                protein_feat_true[get_indexes(m,x),j]=1
        j=j+1
    return protein_feat_true

In [9]:
protein_feat_true=feature_building(protein_b_list_new=protein_b_list_new,protein_a_list_new=protein_a_list_new)
protein_feat_true_pd=pd.DataFrame(protein_feat_true.T)

protein_feat_non_true=feature_building(protein_b_list_new=protein_b_list_non_new,protein_a_list_new=protein_a_list_non_new)
protein_feat_non_true_pd=pd.DataFrame(protein_feat_non_true.T)

### How many ones and twos are in each dataset

In [10]:
index_2_true=protein_feat_true_pd.where(protein_feat_true_pd==2)
index_2_true_count=index_2_true.count(axis=1).sum()

index_1_true=protein_feat_true_pd.where(protein_feat_true_pd==1)
index_1_true_count=index_1_true.count(axis=1).sum()

index_2_nontrue=protein_feat_non_true_pd.where(protein_feat_non_true_pd==2)
index_2_nontrue_count=index_2_nontrue.count(axis=1).sum()

index_1_nontrue=protein_feat_non_true_pd.where(protein_feat_non_true_pd==1)
index_1_nontrue_count=index_1_nontrue.count(axis=1).sum()


print('fraction of twos in the SL array is',index_2_true_count/(len(protein_feat_true_pd.index)*len(protein_feat_true_pd.columns)))
print('fraction of ones in the SL array is',index_1_true_count/(len(protein_feat_true_pd.index)*len(protein_feat_true_pd.columns)))
print('fraction of twos in the PI array is',index_2_nontrue_count/(len(protein_feat_non_true_pd.index)*len(protein_feat_non_true_pd.columns)))
print('fraction of ones in the PI array is',index_1_nontrue_count/(len(protein_feat_non_true_pd.index)*len(protein_feat_non_true_pd.columns)))

fraction of twos in the SL array is 3.0269839689335856e-05
fraction of ones in the SL array is 0.001008463606492084
fraction of twos in the PI array is 9.11369332421964e-06
fraction of ones in the PI array is 0.0009072267445473187


#### Bar plot to visualize these numbers

In [11]:
plt.bar(['fraction of 2 in the nSL','fraction of 1 in the nSL'],[index_2_nontrue_count/(len(protein_feat_non_true_pd.index)*len(protein_feat_non_true_pd.columns)),index_1_nontrue_count/(len(protein_feat_non_true_pd.index)*len(protein_feat_non_true_pd.columns))],alpha=0.6,color=['blue','lightblue']), 

plt.bar(['fraction of 2 in SL ','fraction of 1 in SL'],[index_2_true_count/(len(protein_feat_true_pd.index)*len(protein_feat_true_pd.columns)),index_1_true_count/(len(protein_feat_true_pd.index)*len(protein_feat_true_pd.columns))],alpha=0.6,color=['coral','lightcoral'])

plt.ylabel('Fraction from the population')
plt.yscale('log')
plt.xticks(rotation=40)

([0, 1, 2, 3], <a list of 4 Text major ticklabel objects>)

#### Adding the labels(response variables) to each dataset

In [12]:
protein_feat_true_pd['lethality']=np.ones(shape=(len(protein_a_list_new)))
protein_feat_non_true_pd['lethality']=np.zeros(shape=(len(protein_a_list_non_new)))

#### Joining both datasets

In [13]:
feature_post=pd.concat([protein_feat_true_pd,protein_feat_non_true_pd],axis=0)
feature_post=feature_post.set_index(np.arange(0,len(protein_a_list_new)+len(protein_a_list_non_new)))
print('The number of features are:',feature_post.shape[1])
print('The number of samples are:',feature_post.shape[0])

The number of features are: 3026
The number of samples are: 814


### Postprocessing and exploration of the feature matrix of both datasets

In [14]:
mean=feature_post.T.describe().loc['mean']
std=feature_post.T.describe().loc['std']
lethality=feature_post['lethality']

corr_keys=pd.concat([mean,std,lethality],axis=1)

### Viz of the stats

In [15]:
pair=sns.pairplot(corr_keys,hue='lethality',diag_kind='kde',kind='reg',palette='colorblind')
pair.fig.suptitle('Pairplot to see data dependencies with Lethality',y=1.08)

Text(0.5, 1.08, 'Pairplot to see data dependencies with Lethality')

In [16]:
X, y = feature_post.drop(columns=["lethality"]), feature_post["lethality"]

In [17]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_sl=protein_feat_true_pd.drop(columns=['lethality'])
x_nsl=protein_feat_non_true_pd.drop(columns=['lethality'])

X_sl_scaled = scaler.fit_transform(x_sl)
X_nsl_scaled = scaler.fit_transform(x_nsl)

In [18]:
np.shape(X_sl_scaled)

(415, 3025)

## How redundant are each of the protein domains?

In [19]:
def PCA_component_contribution(scaled_matrix,original_data):
    from sklearn.decomposition import PCA

    model = PCA(0.95).fit(scaled_matrix)
    ## apply dimensionality reduction to X_train
    output_pca = model.transform(scaled_matrix)
    
    total=sum(model.explained_variance_)


   
    # number of components , that it will be the number of main axes times the number of original components 
    n_pcs= model.components_.shape[0] # the amount of non redundant protein domains 

    # get the index of the most important feature on EACH component
    # LIST COMPREHENSION HERE
    most_important = [np.abs(model.components_[i]).argmax() for i in range(n_pcs)]

    initial_feature_names = original_data.columns
    # get the names
    most_important_names = [initial_feature_names[most_important[i]] for i in range(n_pcs)]
    

    # LIST COMPREHENSION HERE AGAIN
    dic = {'PC{}'.format(i): most_important_names[i] for i in range(n_pcs)}

    # build the dataframe
    df = pd.DataFrame(dic.items(),columns=['pca-component','domain-number'])

    return df,model.components_

In [20]:
df_sl,components_pca_nsl=PCA_component_contribution(X_nsl_scaled,x_nsl)
df_nsl,components_pca_sl=PCA_component_contribution(X_sl_scaled,x_sl)

In [21]:
df_sl.head()

pca-component  domain-number
0           PC0           1464
1           PC1            467
2           PC2            135
3           PC3            221
4           PC4            339

## which are the shared domains between nSL and SL pairs , in the PCA reduced samples

In [22]:
df_shared_domains=pd.merge(df_sl,df_nsl,how='inner',on='domain-number')
domains_name=np.unique(data_domains['domain-name'])
domains_description=np.unique(data_domains['domain-descrip'])

df_shared_domains['domain-description']=domains_description[df_shared_domains['domain-number']]


df_shared_domains

pca-component_x  domain-number pca-component_y  \
0              PC7            926            PC18   
1             PC20            877            PC44   
2             PC21           2544            PC13   
3             PC22           2544            PC13   
4             PC23           1836            PC41   
5             PC28            344            PC12   
6             PC29            346           PC124   
7             PC31           1106             PC9   
8             PC43           1162            PC96   
9             PC47           1072            PC77   
10            PC48           1072            PC77   
11            PC51            440            PC28   
12            PC52            440            PC28   
13            PC59           1585            PC80   
14            PC60           1585            PC80   
15            PC61             50           PC100   
16            PC62             50           PC100   
17            PC64             61            PC65   
18            PC66             61            PC65   
19            PC77           1729            PC90   
20            PC82           1046           PC123   
21            PC82           1046           PC127   
22            PC89             34            PC43   
23            PC89             34            PC48   
24            PC90            602            PC81   
25            PC95           1101            PC51   
26            PC97           1352           PC234   
27            PC99           1672           PC238   
28            PC99           1672           PC248   
29           PC126            861           PC121   
30           PC127           2478            PC24   
31           PC130           1842           PC167   
32           PC133           1086           PC146   
33           PC140             39           PC210   
34           PC144           1969           PC207   
35           PC145           1969           PC207   
36           PC149           2371           PC202   
37           PC150            819           PC115   
38           PC151            819           PC115   
39           PC152            819           PC115   
40           PC164            622           PC141   
41           PC178           1498           PC228   
42           PC183           1245           PC222   
43           PC184           2584           PC128   
44           PC193           1731            PC19   
45           PC209            310           PC236   
46           PC209            310           PC247   
47           PC231            493           PC106   
48           PC243            493           PC106   
49           PC255            672           PC265   

                                   domain-description  
0                      Flp_C; Recombinase Flp protein  
1                                       FF; FF domain  
2   Sybindin; Trafficking protein particle complex...  
3   Sybindin; Trafficking protein particle complex...  
4     PTPLA; Protein-tyrosine phosphatase-like, PTPLA  
5        CENP-N; Centromere protein Chl4/mis15/CENP-N  
6                                 CENP-S; CENP-S/Mhf1  
7                                              HEAT_2  
8                 Heme_oxygenase; Haem oxygenase-like  
9     Glycos_transf_1; Glycosyl transferase, family 1  
10    Glycos_transf_1; Glycosyl transferase, family 1  
11  Candida_ALS_N; Agglutinin-like protein, N-term...  
12  Candida_ALS_N; Agglutinin-like protein, N-term...  
13                NPR3; Nitrogen permease regulator 3  
14                NPR3; Nitrogen permease regulator 3  
15                       ACT_7; GATS-like  ACT domain  
16                       ACT_7; GATS-like  ACT domain  
17                                               AGA2  
18                                               AGA2  
19            PEX11; Peroxisomal biogenesis factor 11  
20  Glyco_hydro_18; Glycoside hydrolase family 18,...  
21  Glyco_hydro_18; Glycoside hydrolase family 18,...  
22    AA_ki

In [23]:
print('The number of shared domains between SL and nSL pairs,after PCA, is = ',len(df_shared_domains), 'out of', len(df_sl)+len(df_nsl), 'so the',100*len(df_shared_domains)/(len(df_sl)+len(df_nsl)),'%')

The number of shared domains between SL and nSL pairs,after PCA, is =  50 out of 538 so the 9.293680297397769 %


## Domains exclusive to SL

In [24]:
df_sl_exclusive=pd.merge(df_sl,df_nsl,how='left',on='domain-number')

domains_name=np.unique(data_domains['domain-name'])
domains_description=np.unique(data_domains['domain-descrip'])

df_sl_exclusive['domain-description']=domains_description[df_sl_exclusive['domain-number']]
df_sl_exclusive[pd.isnull(df_sl_exclusive['pca-component_y'])]

pca-component_x  domain-number pca-component_y  \
0               PC0           1464             NaN   
1               PC1            467             NaN   
2               PC2            135             NaN   
3               PC3            221             NaN   
4               PC4            339             NaN   
..              ...            ...             ...   
270           PC266            510             NaN   
271           PC267            883             NaN   
272           PC268            705             NaN   
273           PC269            103             NaN   
274           PC270            103             NaN   

                                    domain-description  
0    Metallophos_2; Calcineurin-like phosphoesteras...  
1            Choline_transpo; Choline transporter-like  
2    ATP-synt_ab_Xtn; ATPsynthase alpha/beta subuni...  
3    Apq12; Nuclear pore assembly and biogenesis pr...  
4                             CDC50; CDC50/LEM3 family  
..                                                 ...  
270                     Cpn10; GroES chaperonin family  
271                                             FIT1_2  
272  DUF3591; Transcription initiation factor TFIID...  
273  ARPC4; Actin-related protein 2/3 complex subun...  
274  ARPC4; Actin-related protein 2/3 complex subun...  

[225 rows x 4 columns]

In [25]:
print('The number of exclusive domains from SL pairs, after PCA, is = ',len(df_sl_exclusive), 'out of', len(df_sl)+len(df_nsl), 'so the',100*len(df_sl_exclusive)/(len(df_sl)+len(df_nsl)),'%')

The number of exclusive domains from SL pairs, after PCA, is =  275 out of 538 so the 51.11524163568773 %


In [26]:
df_nsl_exclusive=pd.merge(df_sl,df_nsl,how='right',on='domain-number')

domains_name=np.unique(data_domains['domain-name'])
domains_description=np.unique(data_domains['domain-descrip'])

df_nsl_exclusive['domain-description']=domains_description[df_nsl_exclusive['domain-number']]
df_nsl_exclusive[pd.isnull(df_nsl_exclusive['pca-component_x'])]

pca-component_x  domain-number pca-component_y  \
50              NaN           1028             PC0   
51              NaN            305             PC1   
52              NaN           1100             PC2   
53              NaN           2113             PC3   
54              NaN            340             PC4   
..              ...            ...             ...   
272             NaN            815           PC258   
273             NaN           1190           PC261   
274             NaN           1772           PC262   
275             NaN           1817           PC263   
276             NaN           2551           PC266   

                                    domain-description  
50                 GatB_Yqey; Asn/Gln amidotransferase  
51                     Brr6_like_C_C; Brl1/Brr6 domain  
52               HBS1_N; HBS1-like protein, N-terminal  
53       Rav1p_C; RAVE complex protein Rav1 C-terminal  
54   CDC73_C; Cell division control protein 73, C-t...  
..                                                 ...  
272                                ERCC4; ERCC4 domain  
273        Hydantoinase_B; Hydantoinase B/oxoprolinase  
274                                PINIT; PINIT domain  
275          PRMT5; PRMT5 arginine-N-methyltransferase  
276  TAF4; Transcription initiation factor TFIID co...  

[227 rows x 4 columns]

In [27]:
print('The number of exclusive domains from non SL pairs, after PCA, is = ',len(df_nsl_exclusive), 'out of', len(df_sl)+len(df_nsl), 'so the',100*len(df_nsl_exclusive)/(len(df_sl)+len(df_nsl)),'%')

The number of exclusive domains from non SL pairs, after PCA, is =  277 out of 538 so the 51.486988847583646 %


## Let see to the domains-number that get repeated after the PCA analysis , and selecting the most explanatory feature by its explained variance.

In [28]:
duplicated_features_sl=df_sl[df_sl.iloc[:,1].duplicated()]
repeated_features_sl=pd.DataFrame()

domains_name=np.unique(data_domains['domain-name'])
domains_description=np.unique(data_domains['domain-descrip'])

repeated_features_sl['domain-name']=domains_name[duplicated_features_sl.iloc[:,1]] 
repeated_features_sl['domain-description']=domains_description[duplicated_features_sl.iloc[:,1]]

duplicated_features_nsl=df_nsl[df_nsl.iloc[:,1].duplicated()]
repeated_features_nsl=pd.DataFrame()

repeated_features_nsl['domain-name']=domains_name[duplicated_features_nsl.iloc[:,1]] 
repeated_features_nsl['domain-description']=domains_description[duplicated_features_nsl.iloc[:,1]]

In [29]:
only_sl_pd=pd.merge(repeated_features_sl,repeated_features_nsl,how='left',on=['domain-name'])
data_only_sl = only_sl_pd[pd.isnull(only_sl_pd['domain-description_y'])]
data_only_sl['domain-description']=data_only_sl['domain-description_x']
data_only_sl.shape,df_sl_exclusive.shape

((45, 4), (275, 4))

In [30]:
only_nsl_pd=pd.merge(repeated_features_sl,repeated_features_nsl,how='right',on=['domain-name'])
data_only_nsl = only_nsl_pd[pd.isnull(only_nsl_pd['domain-description_x'])]
data_only_nsl

domain-name domain-description_x  \
0      PF02359                  NaN   
1      PF08647                  NaN   
2      PF01602                  NaN   
3      PF00082                  NaN   
4      PF00752                  NaN   
5      PF01608                  NaN   
6      PF01608                  NaN   
7      PF03765                  NaN   
8      PF00929                  NaN   
9      PF02037                  NaN   
10     PF03982                  NaN   
11     PF00832                  NaN   
12     PF00071                  NaN   
13     PF03114                  NaN   
14     PF01459                  NaN   
15     PF02724                  NaN   
16     PF08587                  NaN   
17     PF12726                  NaN   
18     PF03126                  NaN   
19     PF03126                  NaN   
20     PF06046                  NaN   
21     PF04098                  NaN   
22     PF12090                  NaN   
23     PF00348                  NaN   
24     PF06839                  NaN   
25     PF07774                  NaN   
26     PF07774                  NaN   
27     PF03820                  NaN   
28     PF09302                  NaN   
29     PF08595                  NaN   
30     PF08737                  NaN   
31     PF12656                  NaN   
32     PF00909                  NaN   
33     PF00909                  NaN   
34     PF00462                  NaN   
35     PF12718                  NaN   
36     PF00628                  NaN   
37     PF06687                  NaN   
38     PF00410                  NaN   
39     PF05216                  NaN   
40     PF01245                  NaN   
41     PF01245                  NaN   
42     PF01843                  NaN   
43     PF10215                  NaN   

                                 domain-description_y  
0                                     FDF; FDF domain  
1   RNA_pol_L_2; DNA-directed RNA polymerase, RBP1...  
2        DUF2611; Protein of unknown function DUF2611  
3     AA_kinase; Aspartate/glutamate/uridylate kinase  
4                   CNH; Citron homology (CNH) domain  
5                                             DUF2781  
6                                             DUF2781  
7                                                Hid1  
8   CT_A_B; Carboxyltransferase domain, subdomain ...  
9   ELFV_dehydrog; Glutamate/phenylalanine/leucine...  
10                  ISN1; IMP-specific 5-nucleotidase  
11  CPDase; 2&apos;,3&apos;-cyclic-nucleotide 3&ap...  
12  AAA_8; Dynein heavy chain, P-loop containing D...  
13  Glyco_hydro_18; Glycoside hydrolase family 18,...  
14  DUF1077; Protein of unknown function DUF1077, ...  
15                    Fmp27_WPPW; FMP27, WPPW  domain  
16  QRPTase_C; Quinolinate phosphoribosyl transfer...  
17  Steroid_dh; 3-oxo-5-alpha-steroid 4-dehydrogen...  
18      Glyco_hydro_31; Glycoside hydrolase family 31  
19      Glyco_hydro_31; Glycoside hydrolase family 31  
20           Nop16; Ribosome biogenesis protein Nop16  
21    Kinetocho_Slk19; Central kinetochore-associated  
22  Sec61_beta; Protein transport protein SecG/Sec...  
23  Afi1; Arf3-interacting protein 1, N-terminal d...  
24  P21-Arc; Actin-related protein 2/3 complex sub...  
25  PI3_PI4_kinase; Phosphatidylinositol 3-/4-kina...  
26  PI3_PI4_kinase; Phosphatidylinositol 3-/4-kina...  
27            Histidinol_dh; Histidinol dehydrogenase  
28                  Rad60-SLD; Rad60/SUMO-like domain  
29                                    R3H; R3H domain  
30                RNase_P_pop3; RNase P, subunit Pop3  
31  Spo7_2_N; Sporulation-specific protein 71, N-t...  
32                                                CTD  
33                                                CTD  
34                                  Arb2; Arb2 domain  
35             Ste50p-SAM; Ste50, sterile alpha motif  
36  C1_1; Protein kinase C-like, phorbol ester/dia...  
37                    Opi1; Transcription factor Opi1  
38            Aminotran_3; Aminotransferase class-III  
39  Mut

In [31]:
shared_domains_pd=pd.merge(repeated_features_sl,repeated_features_nsl,how='inner',on=['domain-name'])
shared_domains_pd


Empty DataFrame
Columns: [domain-name, domain-description_x, domain-description_y]
Index: []

In [32]:
a_sl=x_sl.iloc[:,np.sort(df_sl.iloc[:,1])]
a_sl.describe().loc['mean'].hist(bins=8),
a_sl.describe().loc['std'].hist(bins=8,alpha=0.4)
plt.xlim([0,0.2])

(0.0, 0.2)

In [33]:
a_nsl=x_nsl.iloc[:,np.sort(df_nsl.iloc[:,1])]
a_nsl.describe().loc['mean'].hist(bins=8),
a_nsl.describe().loc['std'].hist(bins=8,alpha=0.4)
plt.xlim([0,0.2])

(0.0, 0.2)

### Proof of concept with *BEM1*

- Gather the domains of SL and nSL pairs from BEM1
- Check if the domains for the SL and nSL pairs are inside the most important domains after PCA.

In [34]:
domains_bem1=data_domains[data_domains['name']=='BEM1']['domain-descrip']
sl_bem1=data_sl[data_sl['gene-query-name']=='BEM1']['gene-target-name']

In [35]:
sl_bem1

2846      BEM2
2847     CDC24
5004      MSB1
13777     SMI1
13778     BNI1
13779     BEM2
14385     SKN7
Name: gene-target-name, dtype: object

In [36]:
data_sl[data_sl['gene-query-name']=='BEM1']

gene-query gene-target gene-query-name gene-target-name  \
2846     YBR200W     YER155C            BEM1             BEM2   
2847     YBR200W     YAL041W            BEM1            CDC24   
5004     YBR200W     YOR188W            BEM1             MSB1   
13777    YBR200W     YGR229C            BEM1             SMI1   
13778    YBR200W     YNL271C            BEM1             BNI1   
13779    YBR200W     YER155C            BEM1             BEM2   
14385    YBR200W     YHR206W            BEM1             SKN7   

                                  gene-query-description  \
2846   SRO1|phosphatidylinositol-3-phosphate-binding ...   
2847   SRO1|phosphatidylinositol-3-phosphate-binding ...   
5004   SRO1|phosphatidylinositol-3-phosphate-binding ...   
13777  SRO1|phosphatidylinositol-3-phosphate-binding ...   
13778  SRO1|phosphatidylinositol-3-phosphate-binding ...   
13779  SRO1|phosphatidylinositol-3-phosphate-binding ...   
14385  SRO1|phosphatidylinositol-3-phosphate-binding ...   

                                 gene-target-description     interaction-type  \
2846                           IPL2|SUP9|TSL1|L000000168  Synthetic Lethality   
2847   CLS4|Rho family guanine nucleotide exchange fa...  Synthetic Lethality   
5004                                          L000001184  Synthetic Lethality   
13777                                    KNR4|L000000909  Synthetic Lethality   
13778                   PPF3|SHE5|formin BNI1|L000000190  Synthetic Lethality   
13779                          IPL2|SUP9|TSL1|L000000168  Synthetic Lethality   
14385  BRY1|POS9|kinase-regulated stress-responsive t...     Dosage Lethality   

            paper-source  
2846   Peterson J (1994)  
2847   Peterson J (1994)  
5004     Bender A (1991)  
13777   Gorelik M (2011)  
13778   Gorelik M (2011)  
13779   Gorelik M (2011)  
14385   Bouquin N (1999)